In [4]:
# Load relevant fucntions from Azure ML 
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

In [5]:
# Load Azure ID from JSON file
import json

azure_id = open('my_id.json', 'r')
mi = json.load(azure_id)
my_id = mi['my_id']

In [7]:
# Create a new Azure workspace
ws = Workspace.create(name="actividad2",
                      subscription_id = my_id,
                      resource_group = "actividad2",
                      location = "centralindia")

Deploying KeyVault with name actividakeyvault052ab362.
Deploying AppInsights with name actividainsights997aa5f7.
Deployed AppInsights with name actividainsights997aa5f7. Took 6.84 seconds.
Deploying StorageAccount with name actividastoragebd12bbe80.
Deployed KeyVault with name actividakeyvault052ab362. Took 24.04 seconds.
Deployed StorageAccount with name actividastoragebd12bbe80. Took 28.52 seconds.
Deploying Workspace with name actividad2.
Deployed Workspace with name actividad2. Took 41.26 seconds.


In [3]:
# If the workspace already exists, load it
ws = Workspace.get(name="actividad2_re", subscription_id = my_id, resource_group = "actividad2_re")

UserErrorException: UserErrorException:
	Message: You are currently logged-in to c65a3ea6-0f7c-400b-8934-5a6dc1705645 tenant. You don't have access to ee4952794-8a80-4b79-98d2-2167b9d3fab1 subscription, please check if it is in this tenant. All the subscriptions that you have access to in this tenant are = 
 [SubscriptionInfo(subscription_name='Visual Studio Enterprise Subscription Cad', subscription_id='90eaa2d9-15dc-4bc3-9c4f-2c580923e179'), SubscriptionInfo(subscription_name='Azure subscription 1', subscription_id='27631026-eb88-4a37-9c9c-aba00c92c056')]. 
 Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "You are currently logged-in to c65a3ea6-0f7c-400b-8934-5a6dc1705645 tenant. You don't have access to ee4952794-8a80-4b79-98d2-2167b9d3fab1 subscription, please check if it is in this tenant. All the subscriptions that you have access

In [8]:
# Load regression model to workspace
mname = "model"
registered_model = Model.register(model_path="reglog.pkl",
                                  model_name=mname,
                                  workspace=ws)

Registering model model


In [9]:
# Create a new environment with needed libraries
virtual_env = Environment("env-2-bankruptcy")
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn'])

In [10]:
# Load scoring script
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )

# Create a new ACI web service configuration
aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=8)

# Deploy the model as a web service
service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\valer\AppData\Local\Temp\ipykernel_26652\693013631.py:11: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [11]:
service.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"


In [12]:
# Generate scoring URI in Azure for model evaluation
scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri3.json", "w")
file.write(scoreuri)
file.close()